# Introduction
In order to do Data Science we have to be able to gather data. Oftentimes we get data from an API, sometimes from a database. This Jupyter notebook we'll guide you through a project that will involve getting data from an API, storing it in a relational database and performing some SQL queries to do some basic data analysis.

Our API of choice for this project will be GitHub Jobs. Besides being simple to use, it could be a useful resource to job-hunt.

## Assignment
Get acquainted with the GitHub Jobs web interface and read through their API page.
Import the Requests package and call the API with the parameters of your choice.
Print the status code to make sure it went well.

In [1]:
import requests
URL = 'https://jobs.github.com/positions.json'
search = 'data science'

parameters = {'description': search}
response = requests.get(URL, params = parameters)
print(response.status_code)


200


In [2]:
print(response.text)

[{"id":"80946cfe-fef7-11e6-967d-80a60ddc736f","created_at":"Thu Mar 02 03:24:39 UTC 2017","title":"Senior Software Engineer, Ruby on Rails","location":"Honolulu","type":"Full Time","description":"<p><strong>Work with an award-winning software team in Honolulu leading projects in fintech, adtech, sustainability and education from New York to Tokyo.</strong> Ikayzo, a creative agency and software development firm, is looking for Senior Ruby on Rails Developers capable of taking complex projects from ideation to release. In this role he/she will be responsible for building high quality, secure and scalable web applications across diverse industries.</p>\n\n<p>Ikayzo offers a competitive compensation package including medical coverage and 24 days of paid time off per annum. This is a local position in Honolulu. We provide relocation assistance for candidates who pass our interview process. We are not currently sponsoring visas or accepting candidates from recruiters for this position.</p>\

## Assignment
Import json and print the whole json response with the json.dumps() function. See what kind of data it contains.
Get the dictionary of the first job and call the method keys() to have a better idea of the data at hand.
Print some of the first job's content, including images and HTML. You might need to import the IPython package.

In [3]:
import json
json_response = response.json()
print(json.dumps(json_response, sort_keys=True, indent=4))

[
    {
        "company": "Ikayzo",
        "company_logo": "http://github-jobs.s3.amazonaws.com/12c7cc7a-fef7-11e6-8a3b-e2264d4928b5.png",
        "company_url": "http://ikayzo.com",
        "created_at": "Thu Mar 02 03:24:39 UTC 2017",
        "description": "<p><strong>Work with an award-winning software team in Honolulu leading projects in fintech, adtech, sustainability and education from New York to Tokyo.</strong> Ikayzo, a creative agency and software development firm, is looking for Senior Ruby on Rails Developers capable of taking complex projects from ideation to release. In this role he/she will be responsible for building high quality, secure and scalable web applications across diverse industries.</p>\n\n<p>Ikayzo offers a competitive compensation package including medical coverage and 24 days of paid time off per annum. This is a local position in Honolulu. We provide relocation assistance for candidates who pass our interview process. We are not currently sponsoring vi

In [5]:
firstjob = json_response[1] #gets dictionary of first job in json object
firstjob.keys() #prints keys in dictionary

dict_keys(['type', 'location', 'created_at', 'company', 'how_to_apply', 'id', 'company_logo', 'url', 'description', 'company_url', 'title'])

In [6]:
from IPython.display import Image
image_url = firstjob['company_logo']
Image(url=image_url)

In [7]:
print(firstjob['company'],
      firstjob['company_logo'],
      firstjob['title'],
      firstjob['type'],
      firstjob['location'],
      firstjob['company_url'],
      sep='\n')

Fielmann Ventures
http://github-jobs.s3.amazonaws.com/3c3e120e-feab-11e6-8a4a-4530df4eebea.png
Senior Software Developer C#
Full Time
Hamburg, Germany
http://www.fielmann-ventures.com


In [8]:
from IPython.core.display import display, HTML
display(HTML(firstjob['description']))

## ## Import SQLite and create a database.
Create 2 tables: company and job.
Make sure you store 'created_at' as a datetime and not a string.

In [10]:
import sqlite3

sqlite_file = 'jobs_db.sqlite' # name of the sqlite database file

# Connecting to the database file
conn = sqlite3.connect(sqlite_file)

In [11]:
conn.execute(
    "CREATE TABLE company ("
        "id INTEGER NOT NULL,"
        "name TEXT NOT NULL,"
        "url TEXT,"
        "logo_url TEXT,"
        "PRIMARY KEY (id),"
        "UNIQUE (url)"
")")

conn.execute(
    "CREATE TABLE job ("
        "id INTEGER NOT NULL,"
        "company_id INTEGER NOT NULL," 
        "title TEXT NOT NULL,"
        "description TEXT,"
        "created_at TIMESTAMP NOT NULL,"
        "location TEXT,"
        "PRIMARY KEY (id),"
        "FOREIGN KEY(company_id) REFERENCES company (id)"
")")

conn.commit()

In [12]:
from sqlite3 import IntegrityError
from datetime import datetime

created_at_pattern = '%a %b %d %X %Z %Y'

def insert_company(name,url,logo_url,connection=conn):
    try:
        conn.execute("INSERT INTO company (name,url,logo_url) VALUES (?, ?, ?)",
                     (name,url,logo_url))
        new_id, = conn.execute("SELECT last_insert_rowid()").fetchone() #create tuple, fetch first item
        conn.commit()
        return new_id
        
    except IntegrityError as err: #catch if unique doesn't work ouit
        print(err)
        print(name)
        
def insert_job(company_id,title,description,location,created_at,connection=conn):
    try:
        conn.execute("INSERT INTO job (company_id,title,description,location,created_at)"
                    "VALUES (?,?,?,?,?)", 
                     (company_id,title,description,location,datetime.strptime(
                    created_at,created_at_pattern))) #convert created_at string to datetime 
        new_id, = conn.execute("SELECT last_insert_rowid()").fetchone() #create tuple, fetch first item
        conn.commit()
        return new_id
        
    except IntegrityError as err: #catch if unique doesn't work ouit
        print(err)
        print(title)

In [13]:
def insert_data(company_name,url,logo_url,created_at,title,description,location):
    transaction = sqlite3.connect(sqlite_file)
    with transaction:
        #insert company
        company_id = insert_company(company_name,url,logo_url,transaction)
        
        #insert job
        job_id = insert_job(company_id,title,description,location,created_at,transaction)
    return job_id

In [14]:
for job in json_response:
    insert_data(
        job['company'],
        job['company_url'],
        job['company_logo'],
        job['created_at'],
        job['title'],
        job['description'],
        job['location']
    
    )

UNIQUE constraint failed: company.url
Moat
NOT NULL constraint failed: job.company_id
Frontend / JavaScript Engineer
UNIQUE constraint failed: company.url
Booking.com BV
NOT NULL constraint failed: job.company_id
Data Scientist - Analytics
UNIQUE constraint failed: company.url
eHealth Africa Systems
NOT NULL constraint failed: job.company_id
Senior Andriod Developer
UNIQUE constraint failed: company.url
Booz Allen Hamilton
NOT NULL constraint failed: job.company_id
Software Engineer, Mid
UNIQUE constraint failed: company.url
Cox Automotive
NOT NULL constraint failed: job.company_id
Java Developer, Enterprise Product Delivery
UNIQUE constraint failed: company.url
Apple Inc.
NOT NULL constraint failed: job.company_id
Sr. Big Data Engineer
UNIQUE constraint failed: company.url
Cox Automotive
NOT NULL constraint failed: job.company_id
Software Architect
UNIQUE constraint failed: company.url
Apple Inc.
NOT NULL constraint failed: job.company_id
Maps Search Principal/Architect
UNIQUE constra

## Assignment
Execute simple SQL queries in each table and make sure everything looks as was intended.
Create queries the following queries:
Find out how many jobs have the phrase "Data Science" and how many have the word "Python" in the description
Find out how many were created in the past 7 days
Find out how many are in California
 

In [15]:
for row in conn.execute('SELECT * FROM company'):
    print('\n {}'.format(tuple(row)))


 (1, 'Ikayzo', 'http://ikayzo.com', 'http://github-jobs.s3.amazonaws.com/12c7cc7a-fef7-11e6-8a3b-e2264d4928b5.png')

 (2, 'Fielmann Ventures', 'http://www.fielmann-ventures.com', 'http://github-jobs.s3.amazonaws.com/3c3e120e-feab-11e6-8a4a-4530df4eebea.png')

 (3, 'ISRIC', 'https://wageningenuniversity.redirect.your-jobresponse.com/w/136031/V819/apply', 'http://github-jobs.s3.amazonaws.com/22c1c370-fe79-11e6-9a33-6cfe8d6f4a42.gif')

 (4, 'LINE', 'http://stackoverflow.com/jobs/companies/line-corporation', 'http://github-jobs.s3.amazonaws.com/7799e336-dbd4-11e6-86ad-9b97e4b8e572.jpg')

 (5, 'UBC', 'http://ecoscope.ubc.ca/job-opportunities/', 'http://github-jobs.s3.amazonaws.com/639fd422-fe06-11e6-925e-14cf6f053342.png')

 (6, 'Moat', 'http://Moat.com', 'http://github-jobs.s3.amazonaws.com/f18b3f34-fe00-11e6-8fd6-3237d03f3de5.png')

 (7, 'Booking.com BV', 'https://www.booking.com/', 'http://github-jobs.s3.amazonaws.com/8b8d7738-fdd3-11e6-84c5-9df564e62219.jpg')

 (8, 'eHealth Africa Syst

In [17]:
for row in conn.execute('SELECT * FROM job'):
    print('\n {}'.format(tuple(row)))


 (1, 1, 'Senior Software Engineer, Ruby on Rails', '<p><strong>Work with an award-winning software team in Honolulu leading projects in fintech, adtech, sustainability and education from New York to Tokyo.</strong> Ikayzo, a creative agency and software development firm, is looking for Senior Ruby on Rails Developers capable of taking complex projects from ideation to release. In this role he/she will be responsible for building high quality, secure and scalable web applications across diverse industries.</p>\n\n<p>Ikayzo offers a competitive compensation package including medical coverage and 24 days of paid time off per annum. This is a local position in Honolulu. We provide relocation assistance for candidates who pass our interview process. We are not currently sponsoring visas or accepting candidates from recruiters for this position.</p>\n\n<p><strong>Job Description</strong></p>\n\n<p>The position of Senior Rails Developer entails software architecture, implementation, developm

In [22]:
cursor = conn.execute('SELECT COUNT() from job')
total, = cursor.fetchone()

cursor = conn.execute("SELECT COUNT() from job WHERE description LIKE '%python%'")
python, = cursor.fetchone()
print("\n There are {} jobs containing 'data science', and {}% of them also list 'python' in the description"
      .format(total,(python/total)*100))


 There are 30 jobs containing 'data science', and 60.0% of them also list 'python' in the description


In [26]:
from datetime import datetime, timedelta

cursor = conn.execute("SELECT COUNT() FROM job WHERE created_at >= :week_ago",
                     {"week_ago": datetime.today() - timedelta(days=7)})
last_week, = cursor.fetchone()

print("\n There were {} jobs posted in the past week" .format(last_week))


 There were 11 jobs posted in the past week


In [37]:
cursor = conn.execute("SELECT COUNT() FROM job WHERE location LIKE '%CA%' OR location LIKE '%California%'")
californiajobs, = cursor.fetchone()

if californiajobs > 0:
    cursor = conn.execute("SELECT name FROM company JOIN job ON company.id = job.company_id "
                          "WHERE location LIKE '%CA%' OR location LIKE '%California%'") #select CA jobs
    
    cacompanies = cursor.fetchall()
    
    if len(cacompanies) == 1:
        print("There was 1 job listed in California, at the company: {}" .format(cacompanies))
    else:
        
        print("There were {} jobs listed in California,at the follow companies:" .format(californiajobs))
        for company, in cacompanies:
            print(company)
else:
    print("There were no jobs posted located in California")

There were 11 jobs listed in California,at the follow companies:
UBC
Stellar Labs, Inc.
NinthDecimal
Apple Inc.
Label Insight
NinthDecimal
Fungible, Inc.
STRAVA
Kemper
QuantumScape Corporation
Rally Health
